In [ ]:
import pandas as pd
from pathlib import Path
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load metrics

In [ ]:
filepath = Path('auroc_eval/outputs')
output_files = filepath.glob('*.json')

## Read output into list of dicts

In [ ]:
output_contents = []
    
for file in output_files:
    
    with open(file) as file:
        text = json.load(file)
        output_contents.append(text)

In [ ]:
auroc = []
auprc = []
brier = []

for test_split in output_contents:
    
    test_size = test_split['labels_train_path'].replace('auroc_eval/splits/test', '')
    test_size = test_size.replace('/artifacts/train_8b/qwen3_embeddings_labels.feather', '')
    test_size = int(test_size)/100
    
    auroc_train = test_split['metrics']['auroc']['train']
    auroc_test = test_split['metrics']['auroc']['test']
    auprc_train = test_split['metrics']['auprc']['train']
    auprc_test = test_split['metrics']['auprc']['test']
    brier_train = test_split['metrics']['brier']['train']
    brier_test = test_split['metrics']['brier']['test']
    
    row_auroc = {'test_size': test_size, 'train AUROC': auroc_train, 'test AUROC': auroc_test}
    auroc.append(row_auroc)
    row_auprc = {'test_size': test_size, 'train AUPRC': auprc_train, 'test AUPRC': auprc_test}
    auprc.append(row_auprc)
    row_brier = {'test_size': test_size, 'train Brier': brier_train, 'test Brier': brier_test}
    brier.append(row_brier)


auroc = pd.DataFrame.from_records(auroc)
auprc = pd.DataFrame.from_records(auprc)
brier = pd.DataFrame.from_records(brier)

In [ ]:
plot = sns.lineplot(auroc, x = 'test_size', y = 'test AUROC')
plot.set_xticks(np.arange(0.1, 1, 0.1))
plt.tight_layout()
plt.savefig('images/auroc_strat.png')
plt.show(plot)